# Resumo do código

### <u>Código que gera os ficheiros para estudo do comportamento da Delta</u>
---
O objectivo é receber dados da Delta e devolver um conjunto de métricas para prever roturas. Devolve um ficheiro que pode entrar no código 1 para juntar aos dados dos ninjas.

---
- Inputs

> __Dados completos da Delta em pastas de ficheiros__ (de azul a verde)
> - Stocks e trânsito, Sellout do dia anterior

                    ou

> __Ficheiro já completo__ (de vermelho a verde)
> - Stocks e trânsito, Sellout do dia anterior

- Outputs

> __Ficheiro com produtos em causa__ em formato Long

> __Métricas novas:__
> - Roturas de Stock e Pré-rotura
> - Sinal
> - Ciclos e Adequação de Stock
> - MSA (média de sellouts 10 dias antes)
> - STK (Stock disponível + trânsito)
> - (Novo) Balanço médio, mediano, liberal e conservador 
> - (Novo) Dias para a rotura de stock e de prateleira


In [27]:
%%time
import pandas as pd
import os
import glob
import numpy as np
import datetime
from IPython.display import Audio

def escrever_csv(dfa, nome): 
    dfa.to_csv(f'C:\\Users\\Chip7\\Desktop\\B&N\\Dados\\2.Delta\\Sonae\\Stocks\\{nome}.csv', index=False)

CPU times: total: 0 ns
Wall time: 0 ns


---

# <font color=blue><u>Ler os ficheiros na pasta</u><font>

- Demora 12 min

In [4]:
%%time
a=0
directoria = 'C:\\Users\\Chip7\\Desktop\\B&N\\Dados\\2.Delta\\Sonae\\Stocks\\Stocks2023\\Outubro\\'

ficheiros = [file for file in os.listdir(directoria)]
dataframes = []

# Iterar pelos ficheiros todos dentro da directória
for file in ficheiros:
    a+=1
    print (a)
    df = pd.read_excel(directoria + file, engine='pyxlsb')
    dataframes.append(df)

original = pd.concat(dataframes, ignore_index=True)
original['DATA'] = pd.to_datetime(original['DATA'], unit='D', origin=datetime.datetime(1899, 12, 30))

freq = 440  # Hz
duration = 1  # seconds
samples = int(duration * 44100)  # 44100 samples per second
t = np.linspace(0, duration, samples, False)
note = np.sin(freq * t * 2 * np.pi)
Audio(note, rate=44100, autoplay=True)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


TypeError: Invalid datetime unit in metadata string "[d]"

# Limitar a Base

In [9]:
%%time
dfTotal = original.loc[:, ['DATA', 'EAN', 'DESC_ARTIGO', 'STORE', "STORE_NAME", 
                           "SOH", "INTRANSIT", "EXPECTED", "PRES_STOCK", "VND (D-1)", "FLUXO"]].copy()

dfTotal = dfTotal.rename(columns = {"VND (D-1)": "SELLOUT_1_Dias_Antes", 
                                    "SOH":"STOCK_1_Dias_Antes",
                                    "INTRANSIT": "INTRANSIT_1_Dias_Antes", 
                                    "EXPECTED": "EXPECTED_1_Dias_Antes", 
                                    "PRES_STOCK": "PRES_STOCK_1_Dias_Antes"})

#dfTotal['DATA'] = pd.to_datetime(dfTotal['DATA'], unit='D', origin=datetime.datetime(1899, 12, 30))

CPU times: total: 62.5 ms
Wall time: 92.8 ms


In [10]:
dfTotal

,DATA,EAN,DESC_ARTIGO,STORE,STORE_NAME,STOCK_1_Dias_Antes,INTRANSIT_1_Dias_Antes,EXPECTED_1_Dias_Antes,PRES_STOCK_1_Dias_Antes,SELLOUT_1_Dias_Antes,FLUXO
0,2023-10-04,5601082006345,CAFÉ DELTA EXPRESSO BAR GRAO 1KG,1,CNT MATOSINHOS,19.0,0,10,16,1,PBL
1,2023-10-04,5601082006345,CAFÉ DELTA EXPRESSO BAR GRAO 1KG,2,CNT AMADORA,21.0,0,0,16,0,PBL
2,2023-10-04,5601082006345,CAFÉ DELTA EXPRESSO BAR GRAO 1KG,3,CNT CASCAIS,23.0,0,0,16,0,PBL
3,2023-10-04,5601082006345,CAFÉ DELTA EXPRESSO BAR GRAO 1KG,4,CNT GAIASHOPPING,31.0,0,0,16,0,PBL
4,2023-10-04,5601082006345,CAFÉ DELTA EXPRESSO BAR GRAO 1KG,5,CNT LEIRIA,27.0,0,0,16,2,PBL
...,...,...,...,...,...,...,...,...,...,...,...
843016,2023-10-30,5609060807281,CHOCOLATE QUENTE CREMOSO CARAMELO 300G,460,CNT ARRABIDA,0.0,0,0,12,0,PBL
843017,2023-10-30,5609060807281,CHOCOLATE QUENTE CREMOSO CARAMELO 300G,927,CNT BARREIRO,0.0,0,12,12,0,PBL
843018,2023-10-30,5607623012905,VB ADEGA MAYOR RES REG ALENT 75CL,3573,CBD CAMPO MAIOR,4.0,0,0,6,0,DIRECTO
843019,2023-10-30,5607623011106,VT DIZERES ADEGA MAYOR ALENTEJO 75CL,3573,CBD CAMPO MAIOR,58.0,0,0,24,1,DIRECTO


# Organizar dados
> Testar a qualidade da leitura

In [14]:
dfTotal = dfTotal.sort_values(by = ["DESC_ARTIGO", "STORE", "DATA"])
dfTotal[["DATA","DESC_ARTIGO","STORE", "SELLOUT", "SELLOUT_1_Dias_Antes"]].iloc[100:120]           #combined_df.loc(combined_df['DESC_ARTIGO'] == "100% SALICÓRNIA MOIDA 45G").head(20)

,DATA,DESC_ARTIGO,STORE,SELLOUT,SELLOUT_1_Dias_Antes
616247,2023-10-23,100% SALICÓRNIA MOIDA 45G,4,0.0,0
647666,2023-10-24,100% SALICÓRNIA MOIDA 45G,4,0.0,0
679084,2023-10-25,100% SALICÓRNIA MOIDA 45G,4,0.0,0
710490,2023-10-26,100% SALICÓRNIA MOIDA 45G,4,0.0,0
741906,2023-10-27,100% SALICÓRNIA MOIDA 45G,4,0.0,0
773321,2023-10-28,100% SALICÓRNIA MOIDA 45G,4,0.0,0
804736,2023-10-29,100% SALICÓRNIA MOIDA 45G,4,0.0,0
835913,2023-10-30,100% SALICÓRNIA MOIDA 45G,4,NaN,0
24159,2023-10-04,100% SALICÓRNIA MOIDA 45G,5,0.0,0
55166,2023-10-05,100% SALICÓRNIA MOIDA 45G,5,0.0,0


> Colunas para sellouts e stocks do dia

In [12]:
dfTotal["SELLOUT_1_Dias_Antes"] = np.where(dfTotal["SELLOUT_1_Dias_Antes"]<0, 0, dfTotal["SELLOUT_1_Dias_Antes"])
dfTotal['SELLOUT'] = dfTotal.groupby(["STORE","EAN"])['SELLOUT_1_Dias_Antes'].shift(-1)
dfTotal['STOCK'] = dfTotal.groupby(["STORE","EAN"])['STOCK_1_Dias_Antes'].shift(-1)
dfTotal['INTRANSIT'] = dfTotal.groupby(["STORE","EAN"])['INTRANSIT_1_Dias_Antes'].shift(-1)
dfTotal['EXPECTED'] = dfTotal.groupby(["STORE","EAN"])['EXPECTED_1_Dias_Antes'].shift(-1)
dfTotal['PRES_STOCK'] = dfTotal.groupby(["STORE","EAN"])['PRES_STOCK_1_Dias_Antes'].shift(-1)

dfTotal["SELLOUT"] = np.where(dfTotal["SELLOUT"]<0, 0, dfTotal["SELLOUT"])
dfTotal["SELLOUT_1_Dias_Antes"] = np.where(dfTotal["SELLOUT_1_Dias_Antes"]<0, 0, dfTotal["SELLOUT_1_Dias_Antes"])

> Ordenar as colunas

In [33]:
Junto.DATA.unique()

array(['2023-06-17', '2023-06-18', '2023-06-19', '2023-06-20',
       '2023-06-21', '2023-06-22', '2023-06-23', '2023-06-24',
       '2023-06-25', '2023-06-26', '2023-06-27', '2023-06-28',
       '2023-06-29', '2023-06-30', '2023-07-01', '2023-07-02',
       '2023-07-03', '2023-07-04', '2023-07-05', '2023-07-06',
       '2023-07-07', '2023-07-08', '2023-07-09', '2023-07-10',
       '2023-07-11', '2023-07-12', '2023-07-13', '2023-07-14',
       '2023-07-15', '2023-07-16', '2023-07-17', '2023-07-18',
       '2023-07-19', '2023-07-20', '2023-07-21', '2023-07-22',
       '2023-07-23', '2023-07-24', '2023-07-25', '2023-07-26',
       '2023-07-27', '2023-07-28', '2023-07-29', '2023-07-30',
       '2023-07-31', '2023-08-01', '2023-08-02', '2023-08-04',
       '2023-08-05', '2023-08-06', '2023-08-07', '2023-08-08',
       '2023-08-09', '2023-08-10', '2023-08-11', '2023-08-12',
       '2023-08-13', '2023-08-14', '2023-08-15', '2023-08-16',
       '2023-08-17', '2023-08-18', '2023-08-19', '2023-

# Escrever

In [42]:
Junto.DATA.unique()

<DatetimeArray>
['2023-06-17 00:00:00', '2023-06-18 00:00:00', '2023-06-19 00:00:00',
 '2023-06-20 00:00:00', '2023-06-21 00:00:00', '2023-06-22 00:00:00',
 '2023-06-23 00:00:00', '2023-06-24 00:00:00', '2023-06-25 00:00:00',
 '2023-06-26 00:00:00',
 ...
 '2023-10-21 00:00:00', '2023-10-22 00:00:00', '2023-10-23 00:00:00',
 '2023-10-24 00:00:00', '2023-10-25 00:00:00', '2023-10-26 00:00:00',
 '2023-10-27 00:00:00', '2023-10-28 00:00:00', '2023-10-29 00:00:00',
 '2023-10-30 00:00:00']
Length: 279, dtype: datetime64[ns]

In [38]:
df = pd.read_csv("C:\\Users\\Chip7\\Desktop\\B&N\\Dados\\2.Delta\\Sonae\\Stocks\\Stocks_Delta_2023_Limpo.csv")
df['DATA'] = pd.to_datetime(df['DATA'])
Junto =  pd.concat([df,dfTotal], ignore_index=True)

In [43]:
%%time
escrever_csv(Junto, "Stocks_Delta_2023_Até_Outubro_Limpo")

CPU times: total: 7.88 s
Wall time: 25.4 s


# <font color=green>Ficheiros Lidos<font>

---

In [17]:
dataframes = [df_2022, df_2023]
df_Fusão = pd.concat(dataframes, ignore_index=True)

df_Fusão = pd.merge(df_Fusão, df_Prophet[['DATA', 'STORE', 'DESC_ARTIGO', 'Prophet']], how="left", on=['DATA', 'STORE', 'DESC_ARTIGO',] )
#dfCopia=df_Fusão.loc[(df_Fusão['DATA'] >= '2022-12-10') ].copy()
dfCopia = df_Fusão.copy()

In [18]:
dfCopia

,DATA,EAN,DESC_ARTIGO,STORE,STORE_NAME,INTRANSIT,EXPECTED,PRES_STOCK,STOCK,STOCK_1_Dias_Antes,SELLOUT,SELLOUT_1_Dias_Antes,Prophet
0,2022-03-23,5600692769695,100% SALICÓRNIA MOIDA 45G,1,CNT MATOSINHOS,0,0,10,23.0,23.0,0.0,0.0,NaN
1,2022-03-24,5600692769695,100% SALICÓRNIA MOIDA 45G,1,CNT MATOSINHOS,0,0,10,22.0,23.0,1.0,0.0,NaN
2,2022-03-25,5600692769695,100% SALICÓRNIA MOIDA 45G,1,CNT MATOSINHOS,0,0,10,22.0,22.0,0.0,1.0,NaN
3,2022-03-26,5600692769695,100% SALICÓRNIA MOIDA 45G,1,CNT MATOSINHOS,0,0,10,22.0,22.0,0.0,0.0,NaN
4,2022-03-27,5600692769695,100% SALICÓRNIA MOIDA 45G,1,CNT MATOSINHOS,0,0,10,22.0,22.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12929551,2023-05-06,5607623011106,VT DIZERES ADEGA MAYOR ALENTEJO 75CL,6089,MDL BEJA RETAIL,0,0,6,5.0,5.0,0.0,0.0,NaN
12929552,2023-05-07,5607623011106,VT DIZERES ADEGA MAYOR ALENTEJO 75CL,6089,MDL BEJA RETAIL,0,0,6,5.0,5.0,0.0,0.0,NaN
12929553,2023-05-08,5607623011106,VT DIZERES ADEGA MAYOR ALENTEJO 75CL,6089,MDL BEJA RETAIL,0,0,6,5.0,5.0,0.0,0.0,NaN
12929554,2023-05-09,5607623011106,VT DIZERES ADEGA MAYOR ALENTEJO 75CL,6089,MDL BEJA RETAIL,0,0,6,5.0,5.0,0.0,0.0,NaN


# <font color=green>Ficheiro Lido<font>

---

# <font color=red>Opcional:</font> Definir produtos em causa

- Produtos específicos

In [20]:
%%time
# Ler ficheiro dos produtos para dataframe
df_produtos = pd.read_csv('D:\\B&N Dados\\Delta\\Piloto\\produtos.txt', header=None)

# Passar para uma lista
produtos = df_produtos[0].tolist()

# Alterar o dataframe para apenas incluir os produtos em causa
dfFinal = dfCopia[dfCopia["DESC_ARTIGO"].isin(produtos)].copy()

CPU times: total: 656 ms
Wall time: 1.01 s


## <font color=red>Fim</font> 

In [21]:
dfFinal.head()

,DATA,EAN,DESC_ARTIGO,STORE,STORE_NAME,INTRANSIT,EXPECTED,PRES_STOCK,STOCK,STOCK_1_Dias_Antes,SELLOUT,SELLOUT_1_Dias_Antes,Prophet
60608,2022-01-01,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,0,48,120,151.0,151.0,0.0,11.0,3.164691
60609,2022-01-02,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,0,48,120,137.0,151.0,14.0,0.0,2.688339
60610,2022-01-03,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,48,0,120,175.0,137.0,10.0,14.0,2.142398
60611,2022-01-04,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,0,0,120,162.0,175.0,13.0,10.0,2.168166
60612,2022-01-05,5609060007087,BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,1,CNT MATOSINHOS,0,48,120,195.0,162.0,15.0,13.0,2.252065


# Colunas de métricas interessantes

> - ROTURA

In [22]:
# Definir coluna de rotura (se stock menor ou igual a 0 e existe Linear)

dfFinal["ROTURA"] = np.where((dfFinal["STOCK"] <= 0) & (dfFinal["PRES_STOCK"] > 0), 1, 0)

> - PRÉ_ROTURA

In [23]:
# Definir coluna de rotura (se stock menor ou igual a 0)

dfFinal["PRE_ROTURA"] = (dfFinal["STOCK"] < dfFinal["PRES_STOCK"]).astype(int)

In [24]:
dfFinal.columns.tolist()

['DATA',
 'EAN',
 'DESC_ARTIGO',
 'STORE',
 'STORE_NAME',
 'INTRANSIT',
 'EXPECTED',
 'PRES_STOCK',
 'STOCK',
 'STOCK_1_Dias_Antes',
 'SELLOUT',
 'SELLOUT_1_Dias_Antes',
 'Prophet',
 'ROTURA',
 'PRE_ROTURA']

# Métricas até 10 dias antes:

- INSTRANSIT
- EXPECTED
- SELLOUT
- CICLOS
- Dias para Rotura
- Adequação

In [25]:
# Quantos dias antes:

diaI=1         #dia inicial
diaF=10        #dia final

> Função

In [26]:
# Função para colunas de dias anteriores
def dias(df, dia, coluna):         #dia é quantos dias antes
    
    #
    a=int(dia)

    valores = df.groupby(['DESC_ARTIGO', 'STORE'])[coluna].transform(lambda x: x.shift(a))
    valores[:a] = np.nan
    
    df.loc[:,'%s_%s_Dias_Antes' % (coluna, a)] = valores

> - SELLOUTS

In [27]:
%%time
# Usar função para sellouts até 10 dias antes

for i in range(diaI+1, diaF+1):
    dias(dfFinal, i, "SELLOUT")

CPU times: total: 8.42 s
Wall time: 10.7 s


> - STOCKS

In [28]:
# Usar função para Stocks até 10 dias antes

for i in range(diaI, diaF+1):
    dias(dfFinal, i, "STOCK")

In [40]:
dfFinal = dfFinal.iloc[:, np.r_[:10,23:32,10:12,14:23,12,13]]

In [30]:
df_columns = dfFinal.columns.tolist()
indexed_columns = [f"{index}: {column}" for index, column in enumerate(df_columns)]
indexed_columns

['0: DATA',
 '1: EAN',
 '2: DESC_ARTIGO',
 '3: STORE',
 '4: STORE_NAME',
 '5: INTRANSIT',
 '6: EXPECTED',
 '7: PRES_STOCK',
 '8: STOCK',
 '9: STOCK_1_Dias_Antes',
 '10: SELLOUT',
 '11: SELLOUT_1_Dias_Antes',
 '12: Prophet',
 '13: ROTURA',
 '14: PRE_ROTURA',
 '15: SELLOUT_2_Dias_Antes',
 '16: SELLOUT_3_Dias_Antes',
 '17: SELLOUT_4_Dias_Antes',
 '18: SELLOUT_5_Dias_Antes',
 '19: SELLOUT_6_Dias_Antes',
 '20: SELLOUT_7_Dias_Antes',
 '21: SELLOUT_8_Dias_Antes',
 '22: SELLOUT_9_Dias_Antes',
 '23: SELLOUT_10_Dias_Antes',
 '24: STOCK_2_Dias_Antes',
 '25: STOCK_3_Dias_Antes',
 '26: STOCK_4_Dias_Antes',
 '27: STOCK_5_Dias_Antes',
 '28: STOCK_6_Dias_Antes',
 '29: STOCK_7_Dias_Antes',
 '30: STOCK_8_Dias_Antes',
 '31: STOCK_9_Dias_Antes',
 '32: STOCK_10_Dias_Antes']

In [42]:
dfOrg = dfFinal.iloc[:, np.r_[:5,6:,5]].copy()

In [47]:
dfFinal.columns.tolist()

['DATA',
 'EAN',
 'DESC_ARTIGO',
 'STORE',
 'STORE_NAME',
 'INTRANSIT',
 'EXPECTED',
 'PRES_STOCK',
 'STOCK',
 'STOCK_1_Dias_Antes',
 'SELLOUT',
 'SELLOUT_1_Dias_Antes',
 'ROTURA',
 'PRE_ROTURA',
 'SELLOUT_2_Dias_Antes',
 'SELLOUT_3_Dias_Antes',
 'SELLOUT_4_Dias_Antes',
 'SELLOUT_5_Dias_Antes',
 'SELLOUT_6_Dias_Antes',
 'SELLOUT_7_Dias_Antes',
 'SELLOUT_8_Dias_Antes',
 'SELLOUT_9_Dias_Antes',
 'SELLOUT_10_Dias_Antes',
 'STOCK_2_Dias_Antes',
 'STOCK_3_Dias_Antes',
 'STOCK_4_Dias_Antes',
 'STOCK_5_Dias_Antes',
 'STOCK_6_Dias_Antes',
 'STOCK_7_Dias_Antes',
 'STOCK_8_Dias_Antes',
 'STOCK_9_Dias_Antes',
 'STOCK_10_Dias_Antes',
 'INTRANSIT_1_Dias_Antes',
 'INTRANSIT_2_Dias_Antes',
 'INTRANSIT_3_Dias_Antes',
 'INTRANSIT_4_Dias_Antes',
 'INTRANSIT_5_Dias_Antes',
 'INTRANSIT_6_Dias_Antes',
 'INTRANSIT_7_Dias_Antes',
 'INTRANSIT_8_Dias_Antes',
 'INTRANSIT_9_Dias_Antes',
 'INTRANSIT_10_Dias_Antes',
 'EXPECTED_1_Dias_Antes',
 'EXPECTED_2_Dias_Antes',
 'EXPECTED_3_Dias_Antes',
 'EXPECTED_4_Dias_Ant

> - INTRANSIT e EXPECTED

In [34]:
# Usar função para Trânsito até 10 dias antes

#dfOrg = dfFinal.iloc[:, np.r_[:5,6:32,5]].copy()
for i in range(diaI, diaF+1):
    dias(dfFinal, i, "INTRANSIT")
    
for i in range(diaI, diaF+1):
    dias(dfFinal, i, "EXPECTED")

> - STK

In [35]:
# STK do dia = soma dos stocks em loja com os stocks em trânsito no próprio dia

dfFinal["STK"] = dfFinal["STOCK"] + dfFinal["INTRANSIT"] + dfFinal["EXPECTED"]

for i in range(diaI, diaF+1):
    dias(dfFinal, i, "STK")

> - MSA

In [36]:
# MSA do dia = média dos sellouts dos 10 dias anteriores ao dia em causa

dfFinal["MSA10"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).mean())
dfFinal["MSA10Dp"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).std())

for i in range(diaI, diaF+1):
    dias(dfFinal, i, "MSA10")

    
dfFinal["MSA20"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=20).mean())
dfFinal["MSA20Dp"] = dfFinal.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=20).std())
  
for i in range(diaI, diaF+1):
    dias(dfFinal, i, "MSA20")

> - CICLOS

In [37]:
# Coluna de Ciclos de reposição

dfFinal["CICLOS"] = dfFinal["STOCK"]/dfFinal["PRES_STOCK"]

for i in range(diaI, diaF+1):
    dias(dfFinal, i, "CICLOS")

# Final2
> - Dias para rotura de Stock

In [ ]:
dfFinal2 = dfFinal.copy()
# Dias para a rotura mas com o Sellout médio (móvel) dos últimos 10 dias 
dfFinal2["Dias_para_Rotura_Stock"] = dfFinal2["STOCK"] / dfFinal2["MSA10"]

for i in range(diaI, diaF+1):
    dias(dfFinal2, i, "Dias_para_Rotura_Stock")

> - Dias para rotura de Linear

In [ ]:
# Definir a métrica: Preslinear / med(Sellouts 10 dias)
dfFinal2['Dias_Duração_Linear'] = dfFinal2["PRES_STOCK"] / dfFinal2["MSA10"]

for i in range(diaI, diaF+1):
    dias(dfFinal2, i, "Dias_Duração_Linear")

> - Adequação de Stock

In [40]:
# Coluna de adequação de stock


dfFinal2["Adequação"]= np.where(dfFinal2["CICLOS"] > 1.1, "Stock Suficiente", 
                      np.where((dfFinal2["CICLOS"] <= 1.1) & (dfFinal2["INTRANSIT"]+dfFinal2["EXPECTED"]+dfFinal2["STOCK"]>=dfFinal2["PRES_STOCK"]), "Stock Insuf c Forn Adequado", 
                      np.where((dfFinal2["CICLOS"] <= 1.1) & (dfFinal2["INTRANSIT"]+dfFinal2["EXPECTED"]+dfFinal2["STOCK"]<dfFinal2["PRES_STOCK"]), "Stock Insuf c Forn Desadequado", 
                      "")))

for i in range(diaI, diaF+1):
    dias(dfFinal2, i, "Adequação")

>- Balance: sellout / soma stock disponível mais transito.

In [23]:
# Colunas de balanço


# Balance do dia = razão entre o sellout médio e o stock para o dia actual
dfFinal2["Balance"] =  dfFinal2["MSA10"] / dfFinal2["STK"]

for i in range(diaI, diaF+1):
    
    
    valores = (i+1) * dfFinal2["Balance"].shift(i)
    valores[:i] = np.nan
    
    dfFinal2.loc[:,'%s_%s_Dias_Antes' % ("Balance", i)] = valores
    
    


>- Balance optimizado

In [24]:


    
dfFinal2["Balance_Optimized"] = np.where((dfFinal2["MSA10Dp"] / dfFinal2["MSA10"]) * 100 > 100, dfFinal2["MSA20"] / dfFinal2["STK"],
                                dfFinal2["MSA10"] / dfFinal2["STK"])   

for i in range(diaI, diaF+1):
    
    
    valores = (i+1) * dfFinal2["Balance_Optimized"].shift(i)
    valores[:i] = np.nan
    
    dfFinal2.loc[:,'%s_%s_Dias_Antes' % ("Balance_Optimized", i)] = valores

> - Mediano é com o mínimo dos ultimos 10 dias

In [25]:
# Coluna de adequação de stock


# MSA_med do dia = mediana dos sellouts dos 10 dias anteriores (exclui o próprio dia)
# dfFinal2["MdSA"] = dfFinal2.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).median())


# Balance do dia = razão entre o sellout médio e o stock para o dia actual
dfFinal2["Balance_Mediano"] =  dfFinal2.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).median()) / dfFinal2["STK"]

for i in range(diaI, diaF+1):
    
    
    valores = (i+1) * dfFinal2["Balance_Mediano"].shift(i)
    valores[:i] = np.nan
    
    dfFinal2.loc[:,'%s_%s_Dias_Antes' % ("Balance_Mediano", i)] = valores



> - Liberal é com o mínimo dos ultimos 10 dias

In [26]:
# Liberal


# MSA do dia = média dos stocks dos 10 dias anteriores (exclui o próprio dia)
#dfFinal2["LSA"] = dfFinal2.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).min())


# Balance do dia = razão entre o sellout médio e o stock para o dia actual
dfFinal2["Balance_Liberal"] =  dfFinal2.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).min()) / dfFinal2["STK"]

for i in range(diaI, diaF+1):
    
    
    valores = (i+1) * dfFinal2["Balance_Liberal"].shift(i)
    valores[:i] = np.nan
    
    dfFinal2.loc[:,'%s_%s_Dias_Antes' % ("Balance_Liberal", i)] = valores



> - Conservador é com o máximo dos ultimos 10 dias

In [27]:
# Conservador

# MSA_max do dia = máximo dos stocks dos 10 dias anteriores (exclui o próprio dia)
#dfFinal2["CSA"] = dfFinal2.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).max())
 
    
# Balance do dia = razão entre o sellout médio e o stock para o dia actual
dfFinal2["Balance_Conservador"] =  dfFinal2.groupby(['DESC_ARTIGO', "STORE"])['SELLOUT'].shift(1).transform(lambda x: x.rolling(window=10).max()) / dfFinal2["STK"]

for i in range(diaI, diaF+1):
    
    # Multiplicar os balances pelo número de dias+1 antes do dia actual
    valores = (i+1) * dfFinal2["Balance_Conservador"].shift(i)
    valores[:i] = np.nan
    
    dfFinal2.loc[:,'%s_%s_Dias_Antes' % ("Balance_Conservador", i)] = valores



In [28]:
dfFinal2[["DATA","STORE","DESC_ARTIGO","STOCK","INTRANSIT","EXPECTED","SELLOUT","Balance","Balance_4_Dias_Antes","Balance_Liberal","Balance_Liberal_4_Dias_Antes","Balance_Mediano_4_Dias_Antes","Balance_Conservador_4_Dias_Antes"]].tail(20)

,DATA,STORE,DESC_ARTIGO,STOCK,INTRANSIT,EXPECTED,SELLOUT,Balance,Balance_4_Dias_Antes,Balance_Liberal,Balance_Liberal_4_Dias_Antes,Balance_Mediano_4_Dias_Antes,Balance_Conservador_4_Dias_Antes
11988342,2023-04-21,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,38.0,0,0,5.0,0.128947,0.568627,0.026316,0.098039,0.490196,1.078431
11988343,2023-04-22,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,33.0,0,0,9.0,0.151515,0.540816,0.030303,0.102041,0.459184,1.122449
11988344,2023-04-23,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,24.0,0,0,5.0,0.233333,0.600000,0.041667,0.111111,0.500000,1.222222
11988345,2023-04-24,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,19.0,0,6,2.0,0.224000,0.756410,0.040000,0.256410,0.641026,1.410256
11988346,2023-04-25,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,17.0,6,6,1.0,0.182759,0.644737,0.034483,0.131579,0.592105,1.315789
11988347,2023-04-26,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,22.0,6,0,0.0,0.157143,0.757576,0.035714,0.151515,0.757576,1.515152
11988348,2023-04-27,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,22.0,6,6,0.0,0.102941,1.166667,0.000000,0.208333,1.041667,2.083333
11988349,2023-04-28,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,28.0,6,18,2.0,0.063462,1.120000,0.000000,0.200000,1.000000,2.000000
11988350,2023-04-29,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,32.0,0,18,5.0,0.062000,0.913793,0.000000,0.172414,0.862069,1.724138
11988351,2023-04-30,9665,CEVADA SOLÚVEL DELTA FRASCO 200G,27.0,0,24,14.0,0.058824,0.785714,0.000000,0.178571,0.803571,1.607143


# Escrever

- 0's e 1's

In [28]:
# Novo dataframe que apenas inclui dias em que existe a 1ª rotura e o dia anterior a essa rotura

# Tirar 1's depois do primeiro
dfFinalLimitado = dfFinal2[~((dfFinal2["ROTURA"] == 1) & (dfFinal2["ROTURA"].shift(1) == 1))].copy()

# Apenas incluir primeira rotura
dfFinalLimitadoRoturas = dfFinal2[((dfFinal2["ROTURA"] == 1) & (dfFinal2["ROTURA"].shift(1) == 0))].copy() #| ((dfFinal["ROTURA"] == 0) & (dfFinal["ROTURA"].shift(-1) == 1))]

- Dias certos

In [30]:
dfFinal2 = dfFinal2.loc[(dfFinal2['DATA'] >= '2023-01-01') ].copy()
df_DiasCertos = dfFinalLimitado.loc[(dfFinalLimitado['DATA'] >= '2023-01-01') ].copy()
df_RoturasDiasCertos = dfFinalLimitadoRoturas.loc[(dfFinalLimitadoRoturas['DATA'] >= '2023-01-01') ].copy()

- Passar para csv

In [29]:
%%time

escrever_csv(dfFinal2, "StocksDelta_2023_10prod_Tudo")
escrever_csv(df_DiasCertos, "StocksDelta_2023_10prod_Limpo")
escrever_csv(df_RoturasDiasCertos, "StocksDelta_2023_10prod_Roturas")

CPU times: total: 2min 15s
Wall time: 2min 58s
